In [23]:
import awswrangler as wr
from datetime import datetime, timedelta
import sagemaker
import boto3
import json
import subprocess
import pandas as pd

now = datetime.now() - timedelta(hours = 4)

# episodes = ["1716761117540","test"] # pageids of episodes to take
# programs = ["1460018486818"] # program pageids of programs to take

In [15]:
# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token, region_name = "eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

credentials =  {
    "boto_session": session,
    "sagemaker_session": sagemaker_session,
    "role": role
}

In [77]:
query = f"""
SELECT
* 
FROM (
SELECT
*
FROM (
SELECT 
*
FROM 
derived_prod.vrtmax_catalog_mediaid_history WHERE year = {now.year} and month = {now.month} and day = {now.day} and hour = {now.hour} 
AND offering_publication_planneduntil > CURRENT_TIMESTAMP
) vrtmax_catalog
LEFT JOIN (
SELECT 
mediacontent_media_id as mediacontent_media_id_duplicate,
array_join(array_agg(subtitle ORDER BY starttc ASC), ' ') as subtitle
FROM (
SELECT 
MIN(subtitle) as subtitle,
mediacontent_media_id,
starttc
FROM (
SELECT 
subtitle,
mediacontent_media_id,
starttc
FROM
(
SELECT 
objid, subtitle
FROM (
SELECT objid, subtitle, 
row_number() OVER (partition by subtitle ORDER BY objid DESC) as rn 
FROM (
SELECT objid, a_vrt_subti2  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti3  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subtit as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s3"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti4 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s4"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_speech as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_sp"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_subti1 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_su"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
)
WHERE rn = 1
) a
LEFT JOIN (
SELECT objid, segmentcontentid, starttc
FROM (
SELECT objid, segmentcontentid, starttc,
row_number() OVER (partition by segmentcontentid ORDER BY objid DESC) as rn 
FROM (
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub3"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub4"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_subt"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
GROUP BY objid, segmentcontentid, starttc
) 
WHERE rn = 1
) b
ON a.objid = b.segmentcontentid
LEFT JOIN (
SELECT objid,  a_vrt_progra ,a_vrt_media_
FROM (
SELECT objid,  a_vrt_progra ,a_vrt_media_,
row_number() OVER (partition by objid ORDER BY a_vrt_media_ DESC) as rn 
FROM "operationaldata_nonsensitive_prod"."depot_dmo_program"  WHERE import_year = 2024 and import_month = 6 and import_day = 9
) WHERE rn = 1
) c
ON b.objid = c.objid
INNER JOIN (
SELECT mediacontent_media_id, 	mediacontent_media_origin_id  
FROM (
SELECT mediacontent_media_id, mediacontent_media_origin_id, row_number() OVER (PARTITION BY mediacontent_media_origin_id   ORDER BY 	contactmoment_ingest_timestamp DESC)  as rn FROM "marketing_prod"."dtr_vd_dms_audio_video_metadata_datariver"
)
WHERE rn = 1
) d
ON c.a_vrt_media_ = d.mediacontent_media_origin_id
)
GROUP BY mediacontent_media_id, starttc
)
GROUP BY mediacontent_media_id
) subtitles  
ON vrtmax_catalog.mediacontent_media_id = subtitles.mediacontent_media_id_duplicate
) intermediary
LEFT JOIN (
SELECT
mediacontent_pageid_duplicate,mediacontent_episode_castlist
FROM (
SELECT 
mediacontent_pageid as mediacontent_pageid_duplicate,mediacontent_episode_castlist,
row_number() over (partition by mediacontent_pageid ORDER BY mediacontent_page_modificationdate desc) as rn
FROM marketing_prod.aem_vrtvideo_datariver
WHERE year = {now.year}  and mediacontent_episode_castlist is not null )
WHERE rn = 1
) castinfo ON intermediary.mediacontent_pageid = castinfo.mediacontent_pageid_duplicate
"""

df = wr.athena.read_sql_query(sql=query, database="derived_prod",boto3_session=credentials["boto_session"])

In [78]:
# drop columns
df = df.drop(['mediacontent_media_id_duplicate','mediacontent_pageid_duplicate'], axis=1)

In [72]:
import numpy as np
df['mediacontent_episode_castlist'] = df['mediacontent_episode_castlist'].fillna("").apply(np.empty(0))

In [76]:
df.mediacontent_episode_castlist.iloc[0]

[]

In [41]:
# cleaning
df["subtitle"] = df["subtitle"].str.replace('\r', '')
df["subtitle"] = df["subtitle"].str.replace('\n', '')
df["subtitle"] = df["subtitle"].str.replace("-","")

# Fields to use
fields_to_embed = [
    "subtitle","mediacontent_page_description_program","mediacontent_page_description","mediacontent_page_editorialtitle_program"
]

metadata_fields_str = [
    "mediacontent_pagetitle_program","mediacontent_pagetitle_season","mediacontent_pagetitle"
    ,"offering_publication_planneduntil","brand_contentbrand","mediacontent_pageurl","mediacontent_imageurl","mediacontent_programimageurl"
]

metadata_fields_nested = [
 "mediacontent_episode_castlist"
]

# convert everythin to string 
str_converst = df[fields_to_embed+metadata_fields_str].astype(str)
str_converst.fillna('', inplace=True)
str_converst = str_converst.replace('<NA>', "")
str_converst = str_converst.replace('"', "")
df = pd.concat([str_converst,df[metadata_fields_nested]], axis=1)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [20]:
""" # take only  rows where at least one of the fields to be embedded is not null
query_condition = []

for field in fields_to_embed:
    query_condition.append("(" + field + "!= ''")

query_condition = " | ".join(query_condition)
df = df.query(query_condition) """

' # take only  rows where at least one of the fields to be embedded is not null\nquery_condition = []\n\nfor field in fields_to_embed:\n    query_condition.append("(" + field + "!= \'\'")\n\nquery_condition = " | ".join(query_condition)\ndf = df.query(query_condition) '

In [6]:
# loading data into vector db requires one field with all text data to embed
df["info_to_embed"] = df[fields_to_embed].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# rest of columns become metadata automatically, so remove the ones you don't want
df = df.drop(['subtitle'], axis=1)

In [7]:
import csv
df.to_csv("vrtmax_catalog.csv",index=False)

In [8]:
# create csv + metadata combinations (for every row)
for index, row in df.iterrows():
    df[df.index == index][fields_to_embed].to_csv("opensearch/" + row["mediacontent_media_id"] + ".csv",index=False)
    metadata = dict()
    metadata["metadataAttributes"] = df[df.index == index][metadata_fields].to_dict("records")[0]

    # Convert and write JSON object to file
    with open("opensearch/" + row["mediacontent_media_id"] + ".metadata.json", "w") as outfile: 
        json.dump(metadata, outfile)

KeyError: "['subtitle'] not in index"